<a href="https://colab.research.google.com/github/abhijithneilabraham/2000Line/blob/master/embeddingtryout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon Jul  1 15:44:07 2019

@author: abhijithneilabraham
"""
!pip install bert_embedding
from getfunctions import *
import re
from keras.utils import to_categorical
from bert_embedding import BertEmbedding
import numpy as np
import re
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense,Embedding,Flatten
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from keras.models import load_model
import pandas as pd
import multiprocessing as mp
import datetime
import concurrent.futures
 
seed=7 #random seed for reproducibility of results 
np.random.seed(seed)
h=pd.read_csv('test1.csv')
x1=[[str(i)] for i in h['identifier']]
y1=[str(i) for i in h['datatype']]
xtrain,ytrain=preprocessing(x1,y1)

len_x=len(xtrain)
i=0
x2=xtrain
y2=ytrain
results_mean=0
batch_len=50
encoder = LabelEncoder()
encoder.fit(ytrain)#fitting numerical labels to corresponding unique strings 
intmap = get_integer_mapping(encoder)#this is for converting back numerical labels to strings
print(intmap)
en_y = encoder.transform(ytrain)
while(i<len_x):
    xtrain=x2[i:i+batch_len]
    ytrain=y2[i:i+batch_len]
    encoded_y=en_y[i:i+batch_len]
    print(xtrain)
    print(str(i)+" number of iterations are over")
    start = datetime.datetime.now()
   
    with concurrent.futures.ProcessPoolExecutor() as executor:
        embedding = executor.map(get_embedding, xtrain)
    emb=[]
    for m in embedding:
      emb.append(m[0][0][1][0])
    end = datetime.datetime.now()
    print("time required for generating features in this iteration=="+str(end-start))
    print(emb[0])
    bert_features=np.array(emb,dtype=object)
    # print(bert_features.shape())
    if(pre_model()==True):
        embed_model=load_model('wordmodel.h5')
    else:
        embed_model=baseline_model()
    embed_model.summary()
    embed_model.fit(bert_features,encoded_y,batch_size=50,epochs=200,verbose=1)
    estimator = KerasClassifier(build_fn=baseline_model, epochs=50, batch_size=5, verbose=1)
    kfold = KFold(n_splits=10, shuffle=True, random_state=seed)#kfold cross validation
    results = cross_val_score(estimator, bert_features, encoded_y, cv=kfold)
    results_mean+=results.mean()
    embed_model.save('wordmodel.h5')
    i=i+batch_len
    
'''
now we are going to predict using the data contained in test.csv
'''
iterations=len_x//batch_len
results_mean=results_mean//iterations
print("Baseline: %.2f%% (%.2f%%)" % (results_mean*100))
h=open('test.csv','r')
pred=get_x(h)
e=target_bert_features(pred)
test=np.array(e)
predictions=embed_model.predict_classes(test)
for j in predictions:
    typename=np.max(j)
    
    print(intmap[typename])#output is printed as string values of corresponding numerical labels
   
    print("------------")



[['isprime'], ['max'], ['carry'], ['prod'], ['res'], ['res_size'], ['temp'], ['exponent'], ['base'], ['prime_numbers'], ['factors'], ['number'], ['num'], ['count'], ['arr'], ['cat'], ['weight'], ['value'], ['capacity'], ['edgeind'], ['graph'], ['vertexnum'], ['edgenum'], ['dist'], ['edges'], ['src'], ['dst'], ['gsrc'], ['lis'], ['dp'], ['eggfloor'], ['result'], ['length'], ['trace'], ['dist1d'], ['int_min'], ['price'], ['stack'], ['top'], ['valid'], ['p1'], ['p2'], ['p3'], ['p4'], ['p5'], ['p6'], ['p7'], ['cout'], ['lenghtline'], ['minr'], ['int8_max'], ['valuetoconvert'], ['hexarray'], ['hexvalues'], ['remainder'], ['binary'], ['var'], ['primes'], ['no'], ['val'], ['startrow'], ['endrow'], ['startcol'], ['endcol'], ['cnt'], ['gcd'], ['counterzeros'], ['handle'], ['sx'], ['sy'], ['mat'], ['color'], ['board'], ['maze'], ['size'], ['soln'], ['currposrow'], ['currposcol'], ['dis'], ['front'], ['mx'], ['inf'], ['gc'], ['neg'], ['rev'], ['root_x'], ['root'], ['root_y'], ['ans'], ['paragraph

ValueError: ignored